In [1]:
import sys
import os

parent_path = os.path.abspath("..")  # Get absolute path of parent directory
if parent_path not in sys.path:  
    sys.path.append(parent_path)  # Add only if it's not already there
    
from dotenv import load_dotenv
load_dotenv("../.env")


import pandas as pd
import openai

from src.models.gpt_models import generate_variant_question, wrong_word_modify

### Read Excel file

In [2]:
question_answer_df = pd.read_excel("../data/question_answer/QA_test.xlsx")

print(question_answer_df)

print("Excel Length: ", len(question_answer_df))

   no                    base_question  Để cây tươi tốt  variant_question
0   1   Tại sao cần bón phân cho cây ?              NaN               NaN
1   2               Đạm Cà Mau ở đâu ?              NaN               NaN
2   3  Cây nhãn chăm sóc như thế nào ?              NaN               NaN
Excel Length:  3


### Loop over to get row

In [3]:
import concurrent.futures

# Wrapper function for parallel processing
def process_row(index, row):
    init_quest_list = generate_variant_question(row["base_question"])
    
    wrong_spelling_quest = wrong_word_modify(str("\n".join(init_quest_list.quest_list[-5:])))

    final_list = str("\n".join(init_quest_list.quest_list[:-5])) + "\n" +  str("\n".join(wrong_spelling_quest.quest_list))
    
    print(final_list)
    
    return index, final_list

# Run in parallel with ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:  # Adjust workers as needed
    futures = {executor.submit(process_row, idx, row): idx for idx, row in question_answer_df.iterrows()}
    for future in concurrent.futures.as_completed(futures):
        idx, variant = future.result()
        #print(variant, type(variant))
        question_answer_df["variant_question"] = question_answer_df["variant_question"].astype(str)
        question_answer_df.at[idx, "variant_question"] = variant

# Save the updated DataFrame to a new Excel file
question_answer_df.to_excel("../data/question_answer/updated_file.xlsx", index=False)

Đạm Cà Mau nằm ở đâu?
Bạn có biết Đạm Cà Mau ở vị trí nào không?
Địa chỉ của Đạm Cà Mau là gì?
Cho tôi hỏi, Đạm Cà Mau tọa lạc ở đâu vậy?
Đạm Cà Mau thì nằm ở khu vực nào?
Hãy cho tôi biết vị trí của Đạm Cà Mau?
Đạm Cà Mau đang ở chỗ nào?
Cà Mau có đạm thì ở đâu nhỉ?
Đạm Cà Mau được đặt tại đâu?
Cụ thể Đạm Cà Mau ở Đâu vậy bạn?
Mình muốn biết Đam Cà Mau nằm tai đâu
Đam Cà Mau co địa diem ở chổ nào khong
Cà Mau có đam ở ku vực nào vậy
Điạ điểm của Đam Cà Mau là ở đâu
Bạn có thể cho biết Đam Cà Mau vị trí là ở đâu khong
Tại sao lại cần phải bón phân cho cây?
Bón phân cho cây có ý nghĩa gì?
Lý do nào khiến chúng ta cần bón phân cho cây trồng?
Tại sao cây cần được bón phân?
Có lý do gì để bón phân cho cây không?
Tại sao việc bón phân cho cây lại quan trọng?
Bón phân cho cây có tác dụng gì?
Có cần phải bón phân cho cây hay không?
Vì sao bón phân là cần thiết cho cây xanh?
Tại sao phải cung cấp phân bón cho cây?
Thế nào là lý dô cần bons phan cho cây
Bón phan cho cây có lơi ích gì
Bón phan c